In [1]:
import sys; sys.path.insert(0,'../..'); sys.path.insert(0,'/Users/ryan/github/lltk')
from lltk import *
from abslithist import *
from abslithist.words import *

In [2]:
path_texts='/Users/ryan/lltk_data/corpora/wimsatt/texts'

In [3]:
!ls {path_texts}

Addison.Spectator106.txt               Hazlitt.LoveOfTheCountry.txt
Addison.Spectator177.txt               Johnson.Idler10.txt
Addison.Spectator267.txt               Johnson.LifeOfPope.txt
Hazlitt.DrydenAndPope.txt              Johnson.Rambler2.txt
Hazlitt.FirstAcquaintanceWithPoets.txt


In [4]:
fn=os.path.join(path_texts,'Johnson.Rambler2.txt')
with open(fn) as f: txt=f.read()

In [5]:
# !pip install spacy
# !spacy download en_core_web_sm
import spacy
nlp = spacy.load('en_core_web_sm')

In [6]:
def find_noun_phrase(sent, target_token):
    # Traverse up to the head of the noun phrase
    while target_token.dep_ not in ("ROOT", "conj") and target_token.head.pos_ in ("NOUN", "PROPN", "PRON"):
        target_token = target_token.head
        break

    # Collect all tokens in the noun phrase
    noun_phrase_tokens = [token for token in target_token.subtree]
    
    # Return the combined text of the noun phrase
    return noun_phrase_tokens

# Example usage
text = "The hunter of the quick brown fox jumps over the lazy dog."
sent = nlp(text)
word = "fox"
noun_phrase = find_noun_phrase(sent, sent[5])
noun_phrase

[the, quick, brown, fox]

In [7]:
doc = nlp('long ease and indulgence')
doc = nlp('There is a new character and a new consequence; a new thrill, a new joy, and sadness')

In [9]:
for sent in doc.sents:
    for w in sent:
        if w.text == 'and':
            print(w.pos_, w.dep_,w)
            break

CCONJ cc and


In [10]:
def get_conjunct_noun_phrases(sent):
    nl=list(sent.noun_chunks)
    o=[]
    done=set()
    for i,n in enumerate(nl):
        if i in done: continue
        no=[n]
        if n.conjuncts:
            for conj in n.conjuncts:
                for i2,n2 in enumerate(nl):
                    if i2 in done: continue
                    if i<i2 and conj.i in {w.i for w in n2}:
                        no.append(n2)
                        done.add(i2)
        if len(no)>1:
            o.append(no)
    return o

In [11]:
# def get_dep_span(sent, tok, deps={'conj','cc','prep'}):
#     head=tok.head
#     foot = None
#     for w in sent:
#         if w.head == tok:
#             foot = w
#     if head and foot:# and sent[head].pos_ == sent[foot].pos_:
#         span=sent[head.i:foot.i+1]
#         return span



# def get_conjunct_words(sent, deps={'conj','cc','prep'}):
#     o=defaultdict(list)
#     for w in sent:
#         print([w,'-->',w.dep_,'-->',w.head])
#         if w.dep_ in deps:
#             v=get_dep_span(sent, w)
#             if v:
#                 o[w]=v
#     #         print('!',w,w.dep_,w.head)
#     #         key=w
#     #         out=[]
#     #         while key.head and key.head not in set(out) and key.dep_ in deps:
#     #             out.extend([key,key.head])
#     #             key=key.head
#     #         o[key].extend(list(set(out)))
#     # for k in o:
#     #     o[k]=[v for v in o[k] if v.dep_ in deps]
#     #     o[k]=sent[min(o[k]).i : max(o[k]).i+1]
#     o=[list(v) for v in o.values()]
#     o.sort(key = lambda v: -len(v))
#     o2=o
#     # done=set()
#     # for v in o:
#     #     if not all(x in done for x in v):
#     #         done.update(set(v))
#     #         print(done)
#     #         o2.append(v)
#     return [
#         detokenize([x.text for x in v])
#         for v in o2
#         if len(v)>1
#     ]

In [12]:
nlp('the')[0].tag_

'DT'

In [13]:
def get_head_foot(sent, tok):
    head=tok.head
    foot = None
    for w in sent:
        if w.head == tok or w.head == tok.head and w.pos_ not in {'PUNCT'}:
            foot = w
    return head,foot

def find_phrase_subtree(sent,tok, stop=None):
    # o=[x for x in sent if (x==tok or x.head == tok)]
    # if stop != None: o=[x for x in o if x.i<stop]
    # print('>>',tok,o)
    o = [x for x in tok.subtree if (stop is None or x.i<stop)]
    o=sent[min(o).i : max(o).i+1]
    print('>>>',tok,o)
    return tuple(o)

def find_phrase_span(sent,tok, stop=None):
    o=[x for x in sent if (x==tok or x.head == tok) and (stop == None or x.i<stop) and x.i>=tok.i]
    o=sent[min(o).i : max(o).i+1]
    print('>>>',tok,o)
    return tuple(o)


def parallel_phrases(phr1,phr2,tok,pos_aggregation = {
        "PROPN": "NOUN",
        "CCONJ": "CONJ",
        "SCONJ": "CONJ",
    }):
    il=[x.i for x in phr1] + [tok.i] + [x.i for x in phr2]
    il2=list(range(phr1[0].i, phr2[-1].i+1))
    print(il)
    print(il2)
    if il == il2:
        phr1=[x for x in phr1 if x.pos_ not in {'PUNCT'}]
        phr2=[x for x in phr2 if x.pos_ not in {'PUNCT'}]
        while phr1 and phr1[0].pos_=='PRON': phr1=phr1[1:]
        while phr2 and phr2[0].pos_=='PRON': phr2=phr2[1:]
        l1=[pos_aggregation.get(x.pos_,x.pos_) for x in phr1]
        l2=[pos_aggregation.get(x.pos_,x.pos_) for x in phr2]
        print()
        print(l1)
        print(l2)
        print()
        if l1==l2:
            return (tuple(phr1),tuple(phr2))
    return None,None

def get_conjunct_words(sent, deps={'cc','prep'}):
    o=[]
    for w in sent:
        print(w.i,w,w.dep_,w.head)
        if w.dep_ in deps:
            head,foot=get_head_foot(sent,w)
            print((head,foot))
            if head and foot:
                headphr,footphr = find_phrase_subtree(sent,head,stop=w.i), find_phrase_subtree(sent,foot)
                headphr,footphr = parallel_phrases(headphr,footphr,w)
                if headphr and footphr:
                    o.append([headphr,w,footphr])
                else:
                    headphr,footphr = find_phrase_span(sent,head,stop=w.i), find_phrase_span(sent,foot)
                    headphr,footphr = parallel_phrases(headphr,footphr,w)
                    if headphr and footphr:
                        o.append([headphr,w,footphr])
                    # if len(headphr)>1:
                        # print(o[-1])
    return o

In [14]:
sent='smoothness without a fair fight is very bad'
sent='gross polish, fruitless grace, but damn foolishness'
sent='The natural flights of the human mind are not from pleasure to pleasure, but from hope to hope.'
# sent='He did not court the candour, but dared the judgment of the reader'
# sent='the breaking in a flower pot, or fall of a china jar.'
# sent='Who knows if the quick brown fox without the lazy gray seal really want what they want?'
get_conjunct_words(nlp(sent))

0 The det flights
1 natural amod flights
2 flights nsubj are
3 of prep flights
(flights, mind)
>>> flights The natural flights
>>> mind the human mind
[0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 5, 6]

['DET', 'ADJ', 'NOUN']
['DET', 'ADJ', 'NOUN']

4 the det mind
5 human amod mind
6 mind pobj of
7 are ROOT are
8 not neg are
9 from prep are
(are, hope)
>>> are The natural flights of the human mind are not
>>> hope from hope to hope
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 16, 17, 18]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
>>> are are not
>>> hope hope
[7, 8, 9, 18]
[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
10 pleasure pobj from
11 to prep from
(from, pleasure)
>>> from from pleasure
>>> pleasure pleasure
[9, 10, 11, 12]
[9, 10, 11, 12]

['ADP', 'NOUN']
['VERB']

>>> from from pleasure
>>> pleasure pleasure
[9, 10, 11, 12]
[9, 10, 11, 12]

['ADP', 'NOUN']
['VERB']

12 pleasure pobj to
13 , punct are
14 but cc are
(are, hope)
>>> are The natural flights of the human mi

ValueError: min() arg is an empty sequence

In [788]:
find_noun_phrase

<function __main__.find_noun_phrase(sent, target_token)>

In [543]:
get_conjunct_noun_phrases(nlp(sent))

[[polish, grace, damn foolishness]]

'The quick brown fox'

In [467]:
# for w in nlp(sent):
#     siblings = [sib for sib in w.head.children if sib.dep_ == 'conj']
#     print(w.i, w, w.dep_, w.head, list(w.subtree), siblings)

! smoothness
[is, [smoothness, is]]
! without
[is, [without, is]]
! a
[is, [a, fight, fight, without, without, is]]
! fight
[is, [fight, without, without, is]]
[smoothness, is, without, is, without, a, fight, is, without, fight, is]
smoothness without a fight is


['smoothness without a fight is']

In [369]:
def get_parallel_noun_phrases(sent):
    pass

In [102]:
n1=nl[0]

In [105]:
n1[0].i

2

In [101]:
nl[3].conjuncts[0].i

12

In [55]:
sent

That the mind of man is never satisfied with the objects immediately before it, but is always breaking away from the present moment, and losing itself in schemes of future felicity; and that we forget the proper use of the time, now in our power, to provide for the enjoyment of that which, perhaps, may never be granted us, has been frequently remarked; and as this practice is a commodious subject of raillery to the gay, and of declamation to the serious, it has been ridiculed with all the pleasantry of wit, and exaggerated with all the amplifications of rhetorick.

In [28]:
corp1 = lltk.Corpus('ecco_tcp')
corp2 = lltk.Corpus('tedjdh')

In [34]:
len(corp1.meta), len(corp2.meta), len(set(corp1.meta.index) & set(corp2.meta.index))

(3101, 3778, 2188)